# Reports

## Schema

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open FSharp.Data
open OSTOR.ClinicalTrials.Reports

open System
open System.IO

### Caris Schema

In [ ]:
let carisSchemaGenerator = XmlSchemaGenerator()

// Caris report xmls
let carisReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "Caris"|])

let sg =
    DirectoryInfo(carisReportXmls).EnumerateFileSystemInfos("*.xml")
        |> Seq.fold (fun (sg: XmlSchemaGenerator) fileInfo -> sg.AddXml(fileInfo.FullName)) carisSchemaGenerator

sg.SaveSchemaToDisk "./src/Reports/data/carisReport.xsd"

## Validations

### Setup

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO

### FMI Validations

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO

open Utilities

let fmiReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "FMI"|])

let fmiResults =
    DirectoryInfo(fmiReportXmls).EnumerateFileSystemInfos("*.xml")
    |> Seq.map (fun filePath -> FoundationMedicine.Report.Xml(filePath.FullName).ReportInput |> FoundationMedicine.Report.validate)

let (reports, errors) = fmiResults |> Seq.toList |> Result.partition

### Caris Validations

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FsToolkit.ErrorHandling"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO

open Utilities

let carisReportXmlsPath = Path.Combine([|Environment.CurrentDirectory; "data"; "Caris"|])

// interpretation -> pathogencity call
// summary -> pathogenic variant
DirectoryInfo(carisReportXmlsPath).EnumerateFileSystemInfos("*.xml")
|> Seq.collect (fun filePath -> Caris.Report.Xml(filePath.FullName).GenomicAlterations)
|> Seq.filter (fun ga -> ga.MolecularConsequence.IsSome)
|> Seq.choose (fun ga -> ga.TranscriptAlterationDetail)
|> Seq.distinct |> Seq.sort

### Tempus Validations

In [ ]:
#r "nuget: Thoth.Json"
#r "nuget: Thoth.Json.Net"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open OSTOR.ClinicalTrials.Reports

open System
open System.IO
open Thoth.Json.Net
open Utilities

let tempusReportJsonsPath = Path.Combine([| Environment.CurrentDirectory; "data"; "Tempus"|])

let jsonResults =
    DirectoryInfo(tempusReportJsonsPath).EnumerateFileSystemInfos("*.json")
    |> Seq.map (fun filePath ->
        let jsonText = File.ReadAllText(filePath.FullName)
        Tempus.Json.deserializeWithError filePath.Name jsonText
    ) |> Seq.toList

let (jsons, errors) = Result.partition jsonResults
// |> Seq.collect (fun results -> results.``Somatic Potentially Actionable Mutations``)
// |> Seq.collect (fun mutation -> mutation.VariantJsons)

jsons |> Seq.map (fun json -> json.Order)

jsons